<a href="https://colab.research.google.com/github/xtian20/Fetch-Data-Challenge/blob/main/Fetch_Data_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Summary**

The following code in this notebook demonstrates how I explored the three datasets (Product, Transaction, and User) to uncover data quality issues.

It also includes SQL queries designed to address both closed-ended and open-ended questions outlined in the data challenge.

And it highlights an interesting data trend observed from the provided data.

# **Imported Packages & Functions**


In [55]:
# Third party library
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import sqlite3

# Local import
from util import *

# **Data Quality Check**

Here are several dimensions I considered in data cleansing:

* **Data type validation**: Ensuring numeric data is correctly formatted and not stored as strings.
* **Handling missing and duplicate data**: Identifying and addressing gaps or redundancies in the dataset.
* **Data accuracy verification**: Ensuring the data aligns with real-world values and satisfies the primary key requirement by maintaining uniqueness.

In [56]:
# Loading three datasets
products = pd.read_csv('PRODUCTS_TAKEHOME.csv')
transaction = pd.read_csv('TRANSACTION_TAKEHOME.csv')
user = pd.read_csv('USER_TAKEHOME.csv')

## **Products Dataset Check**

**Key Data Quality Issues:**

*   **Field CATEGORY4_4** has **92% missing values**, indicating that this level of product categorization may be incomplete or inconsistenly recorded.
*   **57 duplicate rows** are identified. As a temporary solution, these duplicates have been removed to reduce data redundancy.
*   **MANUFACTURER** and **BRAND** fields have approximately **26% missing values**, which may impact brand-level analysis.
*   **3,968 records had missing BARCODE values**, preventing them from serving as unique product identifiers. **These records have been removed** to maintain data integrity.
*   The **BARCODE** field is **not strictly unique**, as some barcodes appear multiple times with different BRAND values. **To address this issue temporarily, records with duplicate barcodes have been removed.** However, further validation is required to determine the correct product-brand association.







**Challenges:**

*   **High missing data in MANUFACTURER and BRAND**:

      1.   With **26% missing values**, analyzing **brand distribution**, **manufacturer trends**, and **product affiliations** becomes challenging.
      2.   This may introduce **bias** or result in an **incomplete** understanding of product performance at the brand level.


*   **Non-unique BARCODE field:**

      1. **Missing BARCODEs (3,968 records removed)**:

        *   Products without a barcode **cannot be mapped to transactions**, making them unusable for purchase analysis.
        *   The removal of these records may lead to **data loss**, but it ensures consistency in product identification.

      2. A single BARCODE appearing under multiple BRAND names may be due to:

        *   **Rebranding**, where the same product is marketed under different brands.
        *   **Data entry errors**, leading to inconsistent labeling.
        *   **Genuine product** variations sharing the same barcode.

      3. Further **data validation and domain knowledge** are required to distinguish between correct and incorrect mappings.







In [57]:
products.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11


### **Datatype Check**

In [58]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845552 entries, 0 to 845551
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CATEGORY_1    845441 non-null  object 
 1   CATEGORY_2    844128 non-null  object 
 2   CATEGORY_3    784986 non-null  object 
 3   CATEGORY_4    67459 non-null   object 
 4   MANUFACTURER  619078 non-null  object 
 5   BRAND         619080 non-null  object 
 6   BARCODE       841527 non-null  float64
dtypes: float64(1), object(6)
memory usage: 45.2+ MB


**Notes:**
1. The **BARCODE** field will **lose leading zeros** if stored as an integer or numeric data type, which may compromise data integrity. To ensure accurate representation and maintain consistency, it should be **stored as a string**.
2. All other fields **have the expected data types** and do not require modifications.

In [59]:
# Stored the BARCODE datatype to string
products = pd.read_csv('PRODUCTS_TAKEHOME.csv', dtype={'BARCODE':str})

In [60]:
products.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,023278011028
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,035000466815
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,0806810850459


### **Missing Data Check**

Field **CATEGORY_4** is **missing data in over 90% of records**, suggesting that this level of product categorization is incomplete or inconsistently recorded—and therefore could be **excluded from the analysis**.

In [61]:
# Identify the missing data
# using the function from util - check_data_missing(df)
check_data_missing(products)

Dataframe exist null values. Let's dive deeper: 
	Column CATEGORY_1 (object) has 0.01% null values.
	Column CATEGORY_2 (object) has 0.17% null values.
	Column CATEGORY_3 (object) has 7.16% null values.
	Column CATEGORY_4 (object) has 92.02% null values.
	Column MANUFACTURER (object) has 26.78% null values.
	Column BRAND (object) has 26.78% null values.
	Column BARCODE (object) has 0.48% null values.


### **Duplicate Data Check**

In [62]:
# Check the duplicate row
# using the function from util - check_fully_duplicated(df)

check_fully_duplicated(products)

There exists 57 fully duplicate.

Duplicated example:


,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
183865,Alcohol,Beer,Lager,American Lager,MOLSONCOORS,COORS LIGHT,NaN
359328,Alcohol,Beer,Lager,American Lager,MOLSONCOORS,COORS LIGHT,NaN
82900,Health & Wellness,Medicines & Treatments,Allergy & Sinus Medicines & Treatments,NaN,HALEON,FLONASE,NaN
410161,Health & Wellness,Medicines & Treatments,Allergy & Sinus Medicines & Treatments,NaN,HALEON,FLONASE,NaN
764322,Health & Wellness,Medicines & Treatments,Allergy & Sinus Medicines & Treatments,NaN,HALEON,FLONASE,NaN
103494,Health & Wellness,Medicines & Treatments,Vitamins & Herbal Supplements,NaN,HALEON,EMERGEN-C,NaN
443703,Health & Wellness,Medicines & Treatments,Vitamins & Herbal Supplements,NaN,HALEON,EMERGEN-C,NaN
756636,Health & Wellness,Medicines & Treatments,Vitamins & Herbal Supplements,NaN,HALEON,EMERGEN-C,NaN
523828,Health & Wellness,Skin Care,Sunscreen,NaN,"BEIERSDORF, INC.",COPPERTONE,NaN
642489,Health & Wellness,Skin Care,Sunscreen,NaN,"BEIERSDORF, INC.",COPPERTONE,NaN


In [63]:
# Remove duplicate rows, retaining the first occurrence.

products.drop_duplicates(keep='first', inplace=True)

### **Data Accuracy Check**


*   **Each record in the Products table is assumed to represent a unique product.**
*   **Therefore, the BARCODE field is expected to be a unique identifier and could serve as the primary key.**

*   **However, data inconsistencies were identified and addressed:**
    *   **3,968 records** have missing BARCODE values, preventing them from serving as unique identifiers. **These records were removed** to maintain data integrity.
    *   **Duplicate BARCODEs** were found, each associated with different BRAND names, suggesting potential issues such as **rebranding**, **misclassification**, **or data entry errors**. **These records were also removed** to ensure that each product has a unique BARCODE.
*   **After these cleaning steps, the dataset now enforces BARCODE uniqueness, improving its reliability for analysis and transactional mapping.**






In [82]:
# check the uniquenes of BARCODE values.

check_partial_duplicated(df=products, columns_to_check=['BARCODE'])

Values in ['BARCODE'] are completely unique.


In [65]:
# Remove rows where 'BARCODE' is null
products = products.dropna(subset=['BARCODE'])
# products = products[products['BARCODE'] != '']

# Now the 'products' DataFrame contains only records with non-empty BARCODE values


In [66]:
# Dive deep into the duplicated barcode '052336919068' and '017000329260'.

selected_barcodes = ['052336919068', '017000329260']
filtered_products = products[products['BARCODE'].isin(selected_barcodes)]

# Display the filtered data
filtered_products


,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
28421,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,SCHWARZKOPF,052336919068
213340,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,SCHWARZKOPF,017000329260
304021,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,GÖT2B,017000329260
709607,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,GÖT2B,052336919068


In [67]:
# Remove rows where 'BARCODE' is '052336919068' or '017000329260'
products = products[~products['BARCODE'].isin(selected_barcodes)]

# Now the 'products' DataFrame does not contain records with those duplicate barcodes.


## **Transactions Dataset Check**

**Key Data Quality Issues:**

*   **BARCODE** has **12% missing values**, which may prevent accurate data joins with the Products table. **Remove records with missing BARCODE values** to ensure accurate transaction mapping and brand/manufacturer trend analysis.
*   **25,389 receipt IDs** have duplicate entries where either **FINAL_QUANTITY** or **FINAL_SALE** is recorded as zero in one row and a valid value in another. M**erge duplicate receipt ID rows**, retaining the nonzero FINAL_QUANTITY and FINAL_SALE values to ensure accurate transaction records.
*   **171 duplicate rows were identified**, indicating potential data redundancy. **Drop duplicate records** to prevent data inflation.
*   **94 receipts have SCAN_DATE earlier than PURCHASE_DATE**, indicating possible data entry errors. **These records have been removed as a temporary solution** to maintain data consistency.
*  **An outlier receipt with an unusually high FINAL_SALE value of 276 was identified**. **Assuming this is unrealistic, the record has been removed** to maintain data accuracy.


**Challenges and Considerations:**

*   **Interpretation of FINAL_QUANTITY & FINAL_SALE in Duplicates:**

    *  When a receipt ID has multiple rows with FINAL_QUANTITY or FINAL_SALE as zero in one row and valid in another, it is unclear:
        *  Should the nonzero value always be retained, or could the zero values indicate **legitimate adjustments** (e.g., partial returns, discounts, or multi-item receipts)?
        *   Does zero FINAL_SALE mean a **free item**, **a fully refunded purchase**, **or an error in data entry**?


*   **Understanding Outlier Transactions:**
    *  Should wholesale receipts be included in the analysis, or are they **outside the scope of typical consumer transactions**?
    *  When transactions have **extremely high FINAL_QUANTITY values**, do they represent **bulk purchases, system errors, or fraudulent activity**?



In [68]:
transaction.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


### **Datatype Check**

In [69]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECEIPT_ID      50000 non-null  object 
 1   PURCHASE_DATE   50000 non-null  object 
 2   SCAN_DATE       50000 non-null  object 
 3   STORE_NAME      50000 non-null  object 
 4   USER_ID         50000 non-null  object 
 5   BARCODE         44238 non-null  float64
 6   FINAL_QUANTITY  50000 non-null  object 
 7   FINAL_SALE      50000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.1+ MB


**Notes:**


1.   The **BARCODE** field should be **converted to a string format** to maintain data integrity, consistent with its representation in the Products table.
2.   The **FINAL_QUANTITY** field contains **data type inconsistencies**, including string values such as 'zero'. This field should be **converted to a numeric format** to enable accurate quantity calculations.
3. The **FINAL_SALE** field is incorrectly recognized as a string. To ensure precise financial calculations, it should be **converted to a numeric format**.
4. The **PURCHASE_DATE** and **SCAN_DATE** fields have data type errors and should be converted to datetime format to ensure accurate date-based operations and comparisons.


In [70]:
# Stored the BARCODE datatype to string
transaction = pd.read_csv('TRANSACTION_TAKEHOME.csv', dtype={'BARCODE':str})

In [71]:
# check what strings include in FINAL_QUANTITY field
transaction['FINAL_QUANTITY'].unique()

array(['1.00', 'zero', '2.00', '3.00', '4.00', '4.55', '2.83', '2.34',
       '0.46', '7.00', '18.00', '12.00', '5.00', '2.17', '0.23', '8.00',
       '1.35', '0.09', '2.58', '1.47', '16.00', '0.62', '1.24', '1.40',
       '0.51', '0.53', '1.69', '6.00', '2.39', '2.60', '10.00', '0.86',
       '1.54', '1.88', '2.93', '1.28', '0.65', '2.89', '1.44', '2.75',
       '1.81', '276.00', '0.87', '2.10', '3.33', '2.54', '2.20', '1.93',
       '1.34', '1.13', '2.19', '0.83', '2.61', '0.28', '1.50', '0.97',
       '0.24', '1.18', '6.22', '1.22', '1.23', '2.57', '1.07', '2.11',
       '0.48', '9.00', '3.11', '1.08', '5.53', '1.89', '0.01', '2.18',
       '1.99', '0.04', '2.25', '1.37', '3.02', '0.35', '0.99', '1.80',
       '3.24', '0.94', '2.04', '3.69', '0.70', '2.52', '2.27'],
      dtype=object)

In [72]:
# Replace 'zero' with 0.00 and convert to float
transaction['FINAL_QUANTITY'] = transaction['FINAL_QUANTITY'].replace('zero', '0.00')
transaction['FINAL_QUANTITY'] = transaction['FINAL_QUANTITY'].astype(float)


In [73]:
# Replace non-numeric values in 'FINAL_SALE' with NaN and then convert to numeric
transaction['FINAL_SALE'] = pd.to_numeric(transaction['FINAL_SALE'], errors='coerce')

# Convert the column to float, the NaNs will be preserved:
transaction['FINAL_SALE'] = transaction['FINAL_SALE'].astype(float)


In [74]:
# Convert 'PURCHASE_DATE' and 'SCAN_DATE' to datetime objects
transaction['PURCHASE_DATE'] = pd.to_datetime(transaction['PURCHASE_DATE'], utc=True, errors='coerce')
transaction['SCAN_DATE'] = pd.to_datetime(transaction['SCAN_DATE'], utc=True, errors='coerce')

### **Duplicate Data Check**

In [83]:
# Check the duplicate row
# using the function from util - check_fully_duplicated(df)


check_fully_duplicated(transaction)

There's no fully duplicated data.


In [76]:
# Remove duplicate rows, retaining the first occurrence.

transaction.drop_duplicates(keep='first', inplace=True)

A total of **25,389 receipt IDs** have duplicate entries. Within these duplicates, some records contain **FINAL_QUANTITY = 0/NaN** in one row while another row has a valid value, or **FINAL_SALE = 0/NaN** in one row while another has a valid amount.

These inconsistencies **inflate the number of receipts and transactions**, leading to **inaccurate sales and quantity calculations**.

This issue likely stems from **data entry or processing errors**, where transaction details were **incorrectly split across multiple rows**.

A potential solution is to **merge duplicate receipt ID rows**, retaining the **nonzero FINAL_QUANTITY and FINAL_SALE values** to ensure accurate transaction records.

In [77]:
transaction.duplicated(subset=['RECEIPT_ID']).sum()

25389

In [78]:
check_partial_duplicated(df=transaction, columns_to_check=['RECEIPT_ID'])

There exist 24440 duplicated values in field ['RECEIPT_ID'].

Duplicated example:


,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21 00:00:00+00:00,2024-08-21 14:19:06.539000+00:00,WALMART,63b73a7f3d310dceeabd4758,015300014978,1.0,NaN
41567,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21 00:00:00+00:00,2024-08-21 14:19:06.539000+00:00,WALMART,63b73a7f3d310dceeabd4758,015300014978,1.0,1.54
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20 00:00:00+00:00,2024-07-20 09:50:24.206000+00:00,ALDI,62c08877baa38d1a1f6c211a,NaN,0.0,1.49
39291,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20 00:00:00+00:00,2024-07-20 09:50:24.206000+00:00,ALDI,62c08877baa38d1a1f6c211a,NaN,1.0,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18 00:00:00+00:00,2024-08-19 15:38:56.813000+00:00,WALMART,60842f207ac8b7729e472020,078742229751,1.0,NaN
25928,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18 00:00:00+00:00,2024-08-19 15:38:56.813000+00:00,WALMART,60842f207ac8b7729e472020,078742229751,1.0,2.54
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18 00:00:00+00:00,2024-06-19 11:03:37.468000+00:00,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,0.0,3.49
41475,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18 00:00:00+00:00,2024-06-19 11:03:37.468000+00:00,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,1.0,3.49
43233,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04 00:00:00+00:00,2024-07-05 15:56:43.549000+00:00,RANDALLS,6193231ae9b3d75037b0f928,047900501183,1.0,5.29
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04 00:00:00+00:00,2024-07-05 15:56:43.549000+00:00,RANDALLS,6193231ae9b3d75037b0f928,047900501183,1.0,NaN


In [79]:
def merge_duplicate_receipts(transaction):
    # Group by all columns except FINAL_QUANTITY and FINAL_SALE
    grouped = transaction.groupby(['RECEIPT_ID', 'PURCHASE_DATE', 'SCAN_DATE', 'STORE_NAME', 'USER_ID', 'BARCODE'])

    # Apply aggregation to handle duplicate receipts
    merged_transaction = grouped.agg(
        {'FINAL_QUANTITY': lambda x: next((val for val in x if val != 0), 0),  # Keep first non-zero FINAL_QUANTITY, or 0 if all are zero
         'FINAL_SALE': lambda x: next((val for val in x if val != 0), 0)}     # Keep first non-zero FINAL_SALE, or 0 if all are zero
    ).reset_index()

    return merged_transaction

# Example usage (assuming 'transaction' DataFrame is already loaded):
transaction = merge_duplicate_receipts(transaction)


### **Missing Data Check**

In [84]:
check_data_missing(transaction)

Dataframe exist null values. Let's dive deeper: 
	Column RECEIPT_ID (object) has no null values.
	Column PURCHASE_DATE (datetime64[ns, UTC]) has no null values.
	Column SCAN_DATE (datetime64[ns, UTC]) has no null values.
	Column STORE_NAME (object) has no null values.
	Column USER_ID (object) has no null values.
	Column BARCODE (object) has no null values.
	Column FINAL_QUANTITY (float64) has no null values.
	Column FINAL_SALE (float64) has 49.91% null values.


**Remove records with missing BARCODE values** to ensure accurate transaction mapping and brand/manufacturer trend analysis.
*   The removal **won't have a major impact** since 88% of records remain intact.
*   This keeps the dataset **clean and fully joinable** with transactions.

In [85]:
# Remove rows where 'BARCODE' is null in the transaction dataframe

transaction = transaction.dropna(subset=['BARCODE'])

### **Data Accuracy Check**

There are **94 receipts** where the **scan date precedes the purchase date**, indicating data discrepancies. As a temporary solution, these records have been removed to maintain data consistency.

In [86]:
# Filter records where SCAN_DATE is later than PURCHASE_DATE
later_scan_dates = transaction[transaction['PURCHASE_DATE'].dt.date > transaction['SCAN_DATE'].dt.date]

# Display the filtered records
later_scan_dates.head()


,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
46,008c1dcc-0f96-4b04-98c8-2a2bb63ef89d,2024-07-21 00:00:00+00:00,2024-07-20 19:54:23.133000+00:00,WALMART,5dc24cdb682fcf1229d04bd6,681131157339,1.0,3.18
406,04a320ed-2903-45e5-8fd7-6eaf08daef32,2024-06-29 00:00:00+00:00,2024-06-28 11:03:31.783000+00:00,DOLLAR GENERAL STORE,62855f67708670299a658035,049000024685,1.0,6.00
438,05023b3d-5f83-47a7-a17c-8e8521d0bc94,2024-09-08 00:00:00+00:00,2024-09-07 22:22:29.903000+00:00,SHOP RITE,666a43c77c0469953bfd9ae0,064144041640,2.0,NaN
597,06ce3da3-a588-4c37-93b4-0b6d11e42704,2024-06-22 00:00:00+00:00,2024-06-21 12:34:15.665000+00:00,BIG LOTS,646f6ffb7a342372c858487e,311111706324,1.0,4.05
768,08d0e78f-3e63-40a3-8eb0-73fdf76da52c,2024-06-22 00:00:00+00:00,2024-06-21 20:50:01.298000+00:00,DOLLAR GENERAL STORE,664cafb6e04f743a096a837e,076808280098,2.0,NaN


In [29]:
# Drop the records where 'SCAN_DATE' is later than 'PURCHASE_DATE'

transaction = transaction[transaction['PURCHASE_DATE'].dt.date <= transaction['SCAN_DATE'].dt.date]

An outlier receipt with an unusually high **FINAL_SALE value of 276** was identified. Assuming this is unrealistic, the record has been removed to maintain data accuracy.

In [30]:
transaction.describe()

,FINAL_QUANTITY,FINAL_SALE
count,21932.000000,10981.000000
mean,1.084948,4.593144
std,1.908542,4.903998
min,0.010000,0.000000
25%,1.000000,1.920000
50%,1.000000,3.190000
75%,1.000000,5.330000
max,276.000000,71.970000


In [31]:
transaction[transaction['FINAL_QUANTITY']==276]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
21813,fe0780d1-2d02-4822-8f12-7056b1814f17,2024-08-09 00:00:00+00:00,2024-08-11 17:52:18.523000+00:00,MAIN STREET MARKET,5d197f9dd08976510c49d0e6,048001353664,276.0,NaN


In [32]:
# Removed FINAL_QUANTITY = 276

transaction = transaction[transaction['FINAL_QUANTITY'] != 276]

## **Users Dataset Check**

**Key Data Quality Issues**:

*  GENDER options should align with real-world identities but contain variations (e.g., "non_binary" vs. "Non-Binary"), leading to inconsistencies **Standardize gender values** by ensuring uniform formatting and consolidating similar responses into a consistent set of categories.


* Identified records where **CREATED_DATE precedes BIRTH_DATE**, indicating **invalid data entry**. **These records have been removed** to maintain data integrity.


* **Approximately 30% of records** have missing values in the LANGUAGE field, which may impact user demographic analysis.



**Challenges and Considerations**:
*  **Potential Duplicate Users:**
    *  Are there cases where multiple accounts belong to the same user (e.g., same BIRTH_DATE, STATE, and LANGUAGE but different IDs)?

In [33]:
user.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


### **Datatype Check**

In [34]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ID            100000 non-null  object
 1   CREATED_DATE  100000 non-null  object
 2   BIRTH_DATE    96325 non-null   object
 3   STATE         95188 non-null   object
 4   LANGUAGE      69492 non-null   object
 5   GENDER        94108 non-null   object
dtypes: object(6)
memory usage: 4.6+ MB


**Notes:**
1. The **CREATED_DATE** and **BIRTH_DATE** fields have data type errors and should be converted to datetime format to ensure accurate date-based operations and comparisons.

In [35]:
# Convert 'CREATED_DATE' and 'BIRTH_DATE' to datetime objects
user['CREATED_DATE'] = pd.to_datetime(user['CREATED_DATE'], errors='coerce')
user['BIRTH_DATE'] = pd.to_datetime(user['BIRTH_DATE'], errors='coerce')


### **Data Accuracy Check**

**ID** field: Represents a unique user identifier, serving as the primary key.

In [36]:
check_partial_duplicated(df=user, columns_to_check=['ID'])

Values in ['ID'] are completely unique.


**STATE** field: Brief abbrievations are all accurate for U.S. states.

In [37]:
user['STATE'].unique()

array(['CA', 'PA', 'FL', 'NC', 'NY', 'IN', nan, 'OH', 'TX', 'NM', 'PR',
       'CO', 'AZ', 'RI', 'MO', 'NJ', 'MA', 'TN', 'LA', 'NH', 'WI', 'IA',
       'GA', 'VA', 'DC', 'KY', 'SC', 'MN', 'WV', 'DE', 'MI', 'IL', 'MS',
       'WA', 'KS', 'CT', 'OR', 'UT', 'MD', 'OK', 'NE', 'NV', 'AL', 'AK',
       'AR', 'HI', 'ME', 'ND', 'ID', 'WY', 'MT', 'SD', 'VT'], dtype=object)

**LANGUAGE** field: Values are accurate for language codes.

In [38]:
user['LANGUAGE'].unique()

array(['es-419', 'en', nan], dtype=object)

**GENDER** field has inconsistencies that need to be addressed:

1. Gender options should align with real-world identities.
2. There are variations in responses (e.g., "non_binary" vs. "Non-Binary"), leading to inconsistencies.


**Solution**: Standardize gender values by ensuring uniform formatting and consolidating similar responses into a consistent set of categories.

In [39]:
user['GENDER'].unique()

array(['female', nan, 'male', 'non_binary', 'transgender',
       'prefer_not_to_say', 'not_listed', 'Non-Binary', 'unknown',
       'not_specified', "My gender isn't listed", 'Prefer not to say'],
      dtype=object)

In [40]:
# Replace values in the 'GENDER' column
user['GENDER'] = user['GENDER'].replace({'Non-Binary': 'non_binary',
                                         'Prefer not to say': 'prefer_not_to_say',
                                         'My gender isn\'t listed': 'not_listed',
                                         'not_specified': 'prefer_not_to_say'})

**CREATED_DATE and BIRTH_DATE** field:

1. Both CREATED_DATE and BIRTH_DATE fall **within a reasonable range** to maintain data accuracy.
2. Identify a record that CREATED_DATE **precedes** the BIRTH_DATE, indicating **invalid data entry**. **These records have been removed** to maintain data integrity.


ID                        | CREATED_DATE | BIRTH_DATE
--------------------------|--------------|-----------
5f31fc048fa1e914d38d6952  | 2020-08-11   | 2020-10-02

In [41]:
# Find the min and max birth date
min_birth_date = user['BIRTH_DATE'].min()
max_birth_date = user['BIRTH_DATE'].max()

# Find the min and max created date
min_created_date = user['CREATED_DATE'].min()
max_created_date = user['CREATED_DATE'].max()

print(f"Min Birth Date: {min_birth_date}. Max Birth Date: {max_birth_date}")
print(f"Min Created Date: {min_created_date}. Max Created Date: {max_created_date}")

# Check if any created date is before the birth date
created_before_birth = user[user['CREATED_DATE'] < user['BIRTH_DATE']]

if not created_before_birth.empty:
    print("\nThere are created dates before birth dates:")
    display(created_before_birth)
else:
    print("\nNo created dates are before birth dates.")


Min Birth Date: 1900-01-01 00:00:00+00:00. Max Birth Date: 2022-04-03 07:00:00+00:00
Min Created Date: 2014-04-18 23:14:55+00:00. Max Created Date: 2024-09-11 17:59:15+00:00

There are created dates before birth dates:


,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
41974,5f31fc048fa1e914d38d6952,2020-08-11 02:01:41+00:00,2020-10-02 15:27:28+00:00,CA,NaN,NaN


In [42]:
# Drop rows where 'CREATED_DATE' is before 'BIRTH_DATE'
user = user[user['CREATED_DATE'] >= user['BIRTH_DATE']]


### **Duplicate Data Check**

In [43]:
check_fully_duplicated(user)

There's no fully duplicated data.


### **Missing Data**

In [44]:
check_data_missing(user)

Dataframe exist null values. Let's dive deeper: 
	Column ID (object) has no null values.
	Column CREATED_DATE (datetime64[ns, UTC]) has no null values.
	Column BIRTH_DATE (datetime64[ns, UTC]) has no null values.
	Column STATE (object) has 2.8% null values.
	Column LANGUAGE (object) has 31.64% null values.
	Column GENDER (object) has 2.38% null values.


# **Data Analysis by SQL**

In [45]:
# Establish a connection to the SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect('mydatabase2.db')

cursor = conn.cursor()

# 'products', 'transaction', and 'user' are your pandas DataFrames
products.to_sql('products', conn, if_exists='replace', index=False)
transaction.to_sql('transactions', conn, if_exists='replace', index=False)
user.to_sql('users', conn, if_exists='replace', index=False)


96324

### **Closed-Ended Questions**

#### **What are the top 5 brands by receipts scanned among users 21 and over?**

In [ ]:
query1 = """
SELECT
    p.CATEGORY_1
  , p.BRAND
  , COUNT(DISTINCT t.RECEIPT_ID) AS num_receipts
FROM transactions as t
JOIN users as u
ON t.USER_ID = u.ID
JOIN products as p
ON t.BARCODE = p.BARCODE
WHERE 1=1
AND BIRTH_DATE <= DATE('now', '-21 years') -- Users aged 21 and over
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 5
"""

top5_brand = pd.read_sql_query(query1, conn)
top5_brand

#### **What are the top 5 brands by sales among users that have had their account for at least six months?**

In [ ]:
query2 = """
SELECT
    p.CATEGORY_1
  , p.BRAND
  , SUM(t.FINAL_SALE) AS total_sales
FROM transactions as t
JOIN users as u
ON t.USER_ID = u.ID
JOIN products as p
ON t.BARCODE = p.BARCODE
WHERE 1=1
AND DATE('now', '-6 months') >= CREATED_DATE -- Users that have had their account for at least six months
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 5
"""


top5_brand_by_sales = pd.read_sql_query(query2, conn)
top5_brand_by_sales

#### **What is the percentage of sales in the Health & Wellness category by generation?**

In [ ]:
query3 = """
WITH CTE AS (
SELECT
    u.ID
  , u.BIRTH_DATE
  , CASE
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1928 AND 1945 THEN 'Silent Generation'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1946 AND 1964 THEN 'Baby Boomers'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1965 AND 1980 THEN 'Gen X'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1981 AND 1996 THEN 'Millennials'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1997 AND 2012 THEN 'Gen Z'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) >= 2013 THEN 'Gen Alpha'
        ELSE 'Unknown'
    END AS generation
  , t.FINAL_SALE
FROM users as u
JOIN transactions as t
ON u.ID = t.USER_ID
JOIN products as p
ON t.BARCODE = p.BARCODE
WHERE p.CATEGORY_1 = 'Health & Wellness'
)
SELECT
    c.generation
  , ROUND(SUM(c.FINAL_SALE)*100 / (SELECT SUM(FINAL_SALE) FROM CTE), 2) AS health_wellness_percentage
FROM CTE as c
GROUP BY 1
ORDER BY 2 DESC
"""


health_wellness_percentage = pd.read_sql_query(query3, conn)
health_wellness_percentage

### **Open-Ended Questions**

#### **Who are Fetch’s power users?**

Fetch is a reward-based platform where users earn points by scanning and uploading receipts. The goal is to identify power users—those who frequently upload receipts, engage consistently over time, and demonstrate long-term retention. These high-engagement users play a critical role in the platform’s success by driving activity, increasing transaction volume, and contributing to overall user retention.

To measure power user behavior, let's first define them as users who **regularly scan and upload receipts**, indicating **high transaction volume**. Therefore, **Total Receipts Uploaded per User per Year** will be the key metric for identifying power users.



In [ ]:
receipts = """
SELECT
    t.USER_ID
  , strftime('%Y', t.SCAN_DATE) AS year
  , COUNT(DISTINCT t.RECEIPT_ID) AS total_receipts_uploaded
FROM transactions as t
GROUP BY 1,2
ORDER BY 2, 3 DESC
"""


power_users = pd.read_sql_query(receipts, conn)
power_users


#### **Which is the leading brand in the Dips & Salsa category?**

To identify the leading brand in the Dips & Salsa category, we first define clearly what "leading" means from a product strategy perspective. In other words, what does success look like for a brand within this category?

We can consider the following product-focused criteria:
1. **Sales Volume (Revenue)**:
    * Identify the brand generating the highest overall sales revenue, indicating strong consumer demand or higher price points.
2. **Purchase Frequency (Unit Sold)**:
    * Find the brand with the most units sold, highlighting strong consumer preference and repeat purchases.
3. **Customer Reach (Number of Unique Customers)**:
    * Determine which brand reaches the broadest audience, indicating widespread popularity or strong brand recognition.
4. **Transaction Presence (Number of Transactions)**:
    * Assess which brand appears most frequently on scanned receipts, suggesting regular consumer engagement.
5. **Consistency Over Time (Stability)**:
    * Identify brands maintaining steady performance across multiple months, demonstrating reliable and sustained consumer loyalty rather than short-term spikes.


Assuming at this time, stores are looking for brands with **strong brand recognition**, the best metrics should focus on **how many unique customers purchase each brand**.

In [ ]:
leading_brand = """
WITH CTE AS (
SELECT
    p.CATEGORY_2
  , p.BRAND
  , COUNT(DISTINCT t.USER_ID) AS unique_customers
  , DENSE_RANK() OVER(ORDER BY COUNT(DISTINCT t.USER_ID)) AS rnk
FROM transactions as t
JOIN products as p
ON t.BARCODE = p.BARCODE
WHERE p.CATEGORY_2 = 'Dips & Salsa'
GROUP BY 1,2
)

SELECT
    cte.CATEGORY_2
  , cte.BRAND
  , cte.unique_customers
FROM CTE
WHERE rnk=1
"""

dips_salsa_brand = pd.read_sql_query(leading_brand, conn)
dips_salsa_brand

#### **At what percent has Fetch grown year over year?**

As a receipt-based rewards platform, Fetch’s success is driven by **user activity**, **transaction volume**, and **engagement levels**. To assess growth, we need to define the KPIs that reflect the platform’s expansion and increasing user value.

Here are four critical dimensions of Fetch’s growth:
1. **User Growth** - Measuring Expansion of the User Base
    *  A larger user base increases data collection, brand partnerships, and revenue potential.
2. **Transaction Growth** – Measuring Usage Frequency
    *  More transactions scanned indicate higher engagement, stronger platform habit-building, and increased data volume for analytics.
3. **Sales Growth** – Measuring Economic Impact
    *  Higher spending levels suggest greater adoption of Fetch by power users, stronger partner brand engagement, and increased monetization opportunities.
4. **Engagement Growth** – Measuring User Retention & Activity
    *  A rising engagement rate indicates that users find value in continuously using the platform, leading to long-term retention and brand loyalty.

Assume Fetch's primary goal is to **scale its user base**, the most important metric is **User Growth**—tracking the growth rate of active users tells us if more people are adopting the platform.

In [ ]:
user_growth = """
WITH active_users_per_year AS (
SELECT
    strftime('%Y', t.SCAN_DATE) AS year
  , COUNT(DISTINCT t.USER_ID) AS active_users
FROM transactions as t
GROUP BY 1
)


SELECT
    au.year as current_year
  , au.active_users as current_active_users
  , au2.active_users as previous_active_users
  , ROUND((au.active_users - au2.active_users)*100/au2.active_users, 2) AS growth_rate
FROM active_users_per_year as au
LEFT JOIN active_users_per_year as au2
ON au.year = au2.year - 1
WHERE au2.year IS NOT NULL
"""

user_growth_yoy = pd.read_sql_query(user_growth, conn)
user_growth_yoy

# Interesting Trend: Weekday vs. Weekend Engagement

- **Observation:** Receipt scans on weekdays significantly outnumber those on weekends.
- **Implication:** This suggests that users are more engaged during the workweek, likely integrating scanning into daily routines (e.g., during commutes or work-related errands).
- **Key Takeaway:** Lower weekend engagement could indicate different shopping habits or less frequent scanning when users are off work.


In [52]:
engagement_receipts = """
SELECT
    strftime('%m', SCAN_DATE) AS month
  , CASE
        WHEN strftime('%w', SCAN_DATE) IN ('0', '1', '2', '3', '4') THEN 'Weekday'
        WHEN strftime('%w', SCAN_DATE) IN ('5', '6') THEN 'Weekend'
    END AS scan_category,
    COUNT(RECEIPT_ID) AS total_receipts_scanned
FROM transactions
GROUP BY 1,2
ORDER BY 1,2

"""

engagement_receipts_monthly = pd.read_sql_query(engagement_receipts, conn)
engagement_receipts_monthly

,month,scan_category,total_receipts_scanned
0,06,Weekday,2833
1,06,Weekend,1373
2,07,Weekday,5913
3,07,Weekend,1972
4,08,Weekday,5270
5,08,Weekend,2549
6,09,Weekday,1520
7,09,Weekend,501


In [53]:
engagement_receipts = """
SELECT
    strftime('%m', SCAN_DATE) AS month
  , CASE
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1928 AND 1945 THEN 'Silent Generation'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1946 AND 1964 THEN 'Baby Boomers'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1965 AND 1980 THEN 'Gen X'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1981 AND 1996 THEN 'Millennials'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) BETWEEN 1997 AND 2012 THEN 'Gen Z'
        WHEN CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) >= 2013 THEN 'Gen Alpha'
        ELSE 'Unknown'
    END AS generation
  , CASE
        WHEN strftime('%w', SCAN_DATE) IN ('0', '1', '2', '3', '4') THEN 'Weekday'
        WHEN strftime('%w', SCAN_DATE) IN ('5', '6') THEN 'Weekend'
    END AS scan_category,
    COUNT(RECEIPT_ID) AS total_receipts_scanned
FROM transactions as t
join users as u
ON t.USER_ID = u.ID
GROUP BY 1,2,3
ORDER BY 2,1,3

"""

engagement_receipts_monthly = pd.read_sql_query(engagement_receipts, conn)
engagement_receipts_monthly

,month,generation,scan_category,total_receipts_scanned
0,06,Baby Boomers,Weekday,5
1,06,Baby Boomers,Weekend,3
2,07,Baby Boomers,Weekday,8
3,07,Baby Boomers,Weekend,6
4,08,Baby Boomers,Weekday,9
5,08,Baby Boomers,Weekend,3
6,09,Baby Boomers,Weekday,2
7,06,Gen X,Weekday,4
8,06,Gen X,Weekend,4
9,07,Gen X,Weekday,5


In [54]:
# Commit the changes and close the connection
conn.commit()
conn.close()